Scraping from News Articles

In [1]:
import os
import json
import time
import random
import zipfile
import requests
import pandas as pd
from bs4 import BeautifulSoup

In [5]:
class ExpressScraper:
    def __init__(self):
        self.id = 0

    def get_express_articles(self, max_pages=9):
        express_df = {
            "id": [],
            "title": [],
            "link": [],
            "content": [],
            "gold_label": []
        }
        base_url = "https://www.express.pk/sports"
        categories = ['saqafat', 'business', 'sports', 'science', 'world']

        # Iterating over the specified number of pages
        for category in categories:
            for page in range(1, max_pages + 1):
                print(f"Scraping page {page} of category '{category}'...")
                url = f"{base_url}/{category}/archives?page={page}"
                
                try:
                    response = requests.get(url)
                    response.raise_for_status()
                    soup = BeautifulSoup(response.text, "html.parser")

                    # Finding article cards
                    cards = soup.find('ul', class_='tedit-shortnews listing-page').find_all('li')
                    print(f"\t--> Found {len(cards)} articles on page {page} of '{category}'.")

                    success_count = 0
                    for card in cards:
                        try:
                            div = card.find("div", class_='horiz-news3-caption')

                            # Article Title
                            headline = div.find('a').get_text(strip=True).replace('\xa0', ' ')
                            # Article Link
                            link = div.find('a')['href']
                            
                            # Requesting the content from each article's link
                            article_response = requests.get(link)
                            article_response.raise_for_status()
                            content_soup = BeautifulSoup(article_response.text, "html.parser")

                            # Content arranged in paras inside <span> tags
                            paras = content_soup.find('span', class_='story-text').find_all('p')
                            combined_text = " ".join([p.get_text(strip=True).replace('\xa0', ' ') for p in paras])

                            # Storing data
                            express_df['id'].append(self.id)
                            express_df['title'].append(headline)
                            express_df['link'].append(link)
                            express_df['gold_label'].append(category.replace('saqafat', 'entertainment').replace('science', 'science-technology'))
                            express_df['content'].append(combined_text)

                            self.id += 1
                            success_count += 1
                        except Exception as e:
                            print(f"\t--> Failed to scrape an article on page {page} of '{category}': {e}")

                    print(f"\t--> Successfully scraped {success_count} articles from page {page} of '{category}'.")
                    print("*" * 50)
                except Exception as e:
                    print(f"Failed to load page {page} of category '{category}': {e}")

        return pd.DataFrame(express_df)

In [7]:
if __name__ == "__main__":
    # Initialize the scraper
    scraper = ExpressScraper()

    # Run the scraper and fetch articles
    try:
        express_df = scraper.get_express_articles(max_pages=10) 
        print("Scraping completed. Sample data:")

        # Display the first few rows of the scraped data

    except Exception as e:
        print("An error occurred during scraping:", e)


Scraping page 1 of category 'saqafat'...
	--> Found 10 articles on page 1 of 'saqafat'.
	--> Successfully scraped 10 articles from page 1 of 'saqafat'.
**************************************************
Scraping page 2 of category 'saqafat'...
	--> Found 10 articles on page 2 of 'saqafat'.
	--> Successfully scraped 10 articles from page 2 of 'saqafat'.
**************************************************
Scraping page 3 of category 'saqafat'...
	--> Found 10 articles on page 3 of 'saqafat'.
	--> Successfully scraped 10 articles from page 3 of 'saqafat'.
**************************************************
Scraping page 4 of category 'saqafat'...
	--> Found 10 articles on page 4 of 'saqafat'.
	--> Successfully scraped 10 articles from page 4 of 'saqafat'.
**************************************************
Scraping page 5 of category 'saqafat'...
	--> Found 10 articles on page 5 of 'saqafat'.
	--> Successfully scraped 10 articles from page 5 of 'saqafat'.
************************************

In [9]:
express_df.head()
express_df

,id,title,link,content,gold_label
0,0,نیٹ فلکس ڈاؤن، سوشل میڈیا پر شکایتوں کا طوفان ...,https://www.express.pk/story/2733439/netflix-d...,معروف امریکی اسٹریمنگ پلیٹ فارم نیٹ فلکس باکسن...,entertainment
1,1,پاکستانی ویب سیریز ’فروٹ چاٹ‘ کی شوٹنگ کا آغا...,https://www.express.pk/story/2733408/the-shoot...,عالمی ایوارڈ یافتہ پاکستان شوبز کے نوجوان ہدای...,entertainment
2,2,ریاض میں رنگارنگ فیشن شو، جینیفر لوپیز سمیت دی...,https://www.express.pk/story/2733380/a-colorfu...,سعودی عرب کےدارالحکومتریاض میں منعقد ہونے والے...,entertainment
3,3,دیشا پٹانی کے والد کے ساتھ لاکھوں روپے کا فراڈ...,https://www.express.pk/story/2733374/disha-pat...,بالی ووڈ کی خوبرو اداکارہ دیشا پٹانی کے والد،...,entertainment
4,4,ڈان 3 میں ولن کون ہوگا؟ نام سامنے آگیا,https://www.express.pk/story/2733372/who-will-...,بالی ووڈ کی مشہور فلم ’ڈان‘ فرنچائز کی تیسری ف...,entertainment
...,...,...,...,...,...
495,495,سوئٹزرلینڈ؛ نقاب پر پابندی اور خلاف ورزی پر سز...,https://www.express.pk/story/2732032/switzerla...,سوئٹزرلینڈ میں 'برقع پر پابندی' کے بل کا اطلاق...,world
496,496,ٹرمپ اسرائیل کی حماس اور لبنان کیساتھ جنگوں کا...,https://www.express.pk/story/2732020/donald-tr...,امریکی صدارتی الیکشن کی فاتح جماعت ریپبلکن کی ...,world
497,497,اسرائیل کی لبنان کے تاریخی شہر پر بمباری؛ 40 ا...,https://www.express.pk/story/2732009/israel-at...,اسرائیل نے لبنان کے مشرقی شہر بعلبک میں تاریخی...,world
498,498,امریکا کی نئی’سیکنڈ لیڈی‘ بھارتی نژاد خاتون ہیں,https://www.express.pk/story/2731989/uselectio...,ڈونلڈ ٹرمپ نے صدارتی الیکشن میں کامیابی کے بعد...,world


In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from time import sleep
from random import randint

class AryNewsScraper:
    def __init__(self):
        self.base_url = 'https://urdu.arynews.tv'
        self.headers = {'User-Agent': 'Mozilla/5.0'}
        self.categories = [
            ('fun-o-sakafat', 'entertainment'),
            ('کاروباری-خبریں', 'business'),
            ('sports-2', 'sports'),
            ('سائنس-اور-ٹیکنالوجی', 'science-technology'),
            ('international-2', 'world')
        ]
        self.id = 1
    
    def fetch_page(self, url):
        """Fetches a single page and handles errors."""
        try:
            response = requests.get(url, headers=self.headers)
            response.raise_for_status()
            return response.text
        except requests.exceptions.RequestException as e:
            print(f"Error fetching page {url}: {e}")
            return None
    
    def parse_article(self, article_url):
        """Fetch and parse the article content."""
        article_response = self.fetch_page(article_url)
        if not article_response:
            return None

        content_soup = BeautifulSoup(article_response, "html.parser")
        paragraphs = content_soup.find('div', class_='vc_column-inner').find_all('p')
        return " ".join([p.get_text(strip=True).replace('\xa0', ' ').replace('\u200b', '') for p in paragraphs if p.get_text(strip=True)])

    def scrape_category(self, category, category_label, max_pages=7):
        """Scrape articles from a given category."""
        ary_data = []
        
        for page in range(1, max_pages + 1):
            print(f"Scraping page {page} of category '{category}'...")
            url = f"{self.base_url}/category/{category}/page/{page}"
            page_html = self.fetch_page(url)
            if not page_html:
                continue

            soup = BeautifulSoup(page_html, "html.parser")
            cards = soup.find_all('div', class_='tdb_module_loop td_module_wrap td-animation-stack td-cpt-post')

            for card in cards:
                try:
                    headline = card.find('a')['title']
                    link = card.find('a')['href']
                    content = self.parse_article(link)

                    if content:  # Only append if content is available
                        ary_data.append({
                            "id": self.id,
                            "title": headline,
                            "link": link,
                            "content": content,
                            "gold_label": category_label
                        })
                        self.id += 1
                except Exception as e:
                    print(f"Error processing article on page {page}: {e}")

            sleep(randint(1, 3))  # Random sleep to avoid rate limiting

        return ary_data

    def scrape_all(self, max_pages=7):
        """Scrape all categories and compile the data."""
        all_data = []
        for category, label in self.categories:
            data = self.scrape_category(category, label, max_pages)
            all_data.extend(data)
        return pd.DataFrame(all_data)

scraper = AryNewsScraper()
articles_df = scraper.scrape_all(max_pages=5)
print(articles_df.head())


Scraping page 1 of category 'fun-o-sakafat'...
Scraping page 2 of category 'fun-o-sakafat'...
Scraping page 3 of category 'fun-o-sakafat'...
Scraping page 4 of category 'fun-o-sakafat'...
Scraping page 5 of category 'fun-o-sakafat'...
Scraping page 1 of category 'کاروباری-خبریں'...
Scraping page 2 of category 'کاروباری-خبریں'...
Scraping page 3 of category 'کاروباری-خبریں'...
Scraping page 4 of category 'کاروباری-خبریں'...
Scraping page 5 of category 'کاروباری-خبریں'...
Scraping page 1 of category 'sports-2'...
Scraping page 2 of category 'sports-2'...
Scraping page 3 of category 'sports-2'...
Scraping page 4 of category 'sports-2'...
Scraping page 5 of category 'sports-2'...
Scraping page 1 of category 'سائنس-اور-ٹیکنالوجی'...
Scraping page 2 of category 'سائنس-اور-ٹیکنالوجی'...
Scraping page 3 of category 'سائنس-اور-ٹیکنالوجی'...
Scraping page 4 of category 'سائنس-اور-ٹیکنالوجی'...
Scraping page 5 of category 'سائنس-اور-ٹیکنالوجی'...
Scraping page 1 of category 'international-2'...
S

In [9]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

class NewsScraper2:
    def __init__(self):
        self.id = 0  # Initialize the article id
        
    def get_dunya_articles(self, max_pages=7):
        dunya_df = {
            "id": [],
            "title": [],
            "link": [],
            "content": [],
            "gold_label": [],
        }
        
        base_url = 'https://urdu.dunyanews.tv'
        categories = ['Entertainment', 'Business', 'Sports', 'Technology', 'World']
        
        # Start scraping categories with a while loop
        cat_index = 0
        while cat_index < len(categories):
            category = categories[cat_index]
            url = f"{base_url}/index.php/ur/{category}"
            response = requests.get(url)
            response.raise_for_status()
            soup = BeautifulSoup(response.text, "html.parser")
        
            cards = soup.find('div', class_='newsBox categories').find_all('div', class_='cNewsBox') 
            print(f"\t--> Found {len(cards)} articles on '{category}'.")
            success_count = 0
            article_index = 0
            
            while article_index < len(cards):
                try:
                    card = cards[article_index]
                    div = card.find('div', class_='col-md-8')
                    
                    headline = div.find('a').get_text(strip=True).replace('\xa0', ' ')
                    link = div.find('a')['href']
                    targetlink = base_url + link
                    
                    article_response = requests.get(targetlink)
                    article_response.raise_for_status()
                    content_soup = BeautifulSoup(article_response.text, "html.parser")
                    
                    paras = content_soup.find('div', class_='main-news col-md-12').find_all('p')
                    lines = ''
                    para_index = 0
                    while para_index < len(paras):
                        p = paras[para_index]
                        lines += p.get_text(strip=True).replace('\xa0', ' ').replace('\u200b', '')
                        para_index += 1
                    combined_text = "".join(lines)
                    
                    dunya_df['id'].append(self.id)
                    dunya_df['title'].append(headline)
                    dunya_df['link'].append(targetlink)
                    dunya_df['gold_label'].append(category.replace('Technology','Science-Technology'))
                    dunya_df['content'].append(combined_text)
                   
                    self.id += 1
                    success_count += 1
                except Exception as e:
                    print(f"\t--> Failed to scrape an article on '{category}': {e}")
                    
                article_index += 1

            print(f"\t--> Successfully scraped {success_count} articles of '{category}'.")
            cat_index += 1

        return pd.DataFrame(dunya_df)
if __name__ == "__main__":
    # Initialize the scraper
    scraper = NewsScraper2()

    # Run the scraper and fetch articles
    try:
        dunya_df = scraper.get_dunya_articles(max_pages=10)
        print("Scraping completed. Sample data:")
        
        # Display the first few rows of the scraped data
        print(dunya_df.head())
        
    except Exception as e:
        print("An error occurred during scraping:", e)


	--> Found 18 articles on 'Entertainment'.
	--> Successfully scraped 18 articles of 'Entertainment'.
	--> Found 18 articles on 'Business'.
	--> Successfully scraped 18 articles of 'Business'.
	--> Found 18 articles on 'Sports'.
	--> Successfully scraped 18 articles of 'Sports'.
	--> Found 18 articles on 'Technology'.
	--> Successfully scraped 18 articles of 'Technology'.
	--> Found 18 articles on 'World'.
	--> Successfully scraped 18 articles of 'World'.
Scraping completed. Sample data:
   id                                              title  \
0   0  عزیز میاں قوال کو دنیا سے رخصت ہوئے 24 برس بیت...   
1   1  ماضی کے مقبول گلوکار اور معروف نعت خواں جنید ج...   
2   2  یونیسیف کی سفیر برائے اطفال صبا قمر کی جرمن قو...   
3   3  بالی وڈ اداکار آدتیہ پنچولی کا بعد ازموت جسم ع...   
4   4  مایا علی کے مایوں لک میں حسن کے جلوے، مداح دل ...   

                                                link  \
0  https://urdu.dunyanews.tv/index.php/ur/Enterta...   
1  https://urdu.dunyanews.tv/index.

In [3]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

class NewsScraper:
    def __init__(self):
        pass

    
    def get_jang_articles(self):
        jang_data = {
            "article_id": [],
            "headline": [],
            "url": [],
            "body_content": [],
            "category_label": []
        }

        base_url = "https://jang.com.pk/category/latest-news"
        jang_sections = {
            'showbiz': 'entertainment',
            'markets': 'business',
            'fitness': 'sports',
            'tech': 'technology',
            'international': 'world'
        }

        section_index = 0
        section_keys = list(jang_sections.keys())
        
        while section_index < len(section_keys):
            section_key = section_keys[section_index]
            category = jang_sections[section_key]
            url = f"{base_url}/{section_key}"
            print(f"Scraping '{category}' section...")
            
            try:
                response = requests.get(url)
                response.raise_for_status()
                soup = BeautifulSoup(response.text, "html.parser")
                articles = soup.find("ul", class_="scrollPaginationNew__").find_all("li")
                article_index = 0
                
                while article_index < len(articles):
                    article = articles[article_index]
                    try:
                        link_tag = article.find("a")
                        article_url = link_tag["href"] if link_tag else None
                        headline = link_tag.get("title", "").strip() if link_tag else None
                        article_id = article.get("data-id", None)

                        if article_url:
                            article_response = requests.get(article_url)
                            article_response.raise_for_status()
                            content_soup = BeautifulSoup(article_response.text, "html.parser")
                            paragraphs = content_soup.find_all("p")
                            body_content = " ".join(p.get_text(strip=True) for p in paragraphs)
                        else:
                            body_content = ""

                        jang_data["article_id"].append(article_id)
                        jang_data["headline"].append(headline)
                        jang_data["url"].append(article_url)
                        jang_data["body_content"].append(body_content)
                        jang_data["category_label"].append(category)

                    except Exception as e:
                        print(f"\t--> Failed to scrape article: {e}")

                    article_index += 1

            except Exception as e:
                print(f"Failed to fetch '{category}', error: {e}")

            section_index += 1

        return pd.DataFrame(jang_data)

    def get_geo_articles(self, max_pages=7):
        geo_data = {
            "article_id": [],
            "headline": [],
            "url": [],
            "body_content": [],
            "category_label": []
        }
        base_url = 'https://urdu.geo.tv'
        geo_sections = ['entertainment', 'business', 'sports', 'science-technology', 'world']

        section_index = 0
        while section_index < len(geo_sections):
            section = geo_sections[section_index]
            url = f"{base_url}/category/{section}"
            print(f"Scraping '{section}' section...")
            
            try:
                response = requests.get(url)
                response.raise_for_status()
                soup = BeautifulSoup(response.text, "html.parser")
                articles = soup.find('div', class_='video-list laodMoreCatNews').find_all(
                    'div', class_='col-xs-6 col-sm-6 col-lg-6 col-md-6 singleBlock'
                )
                article_index = 0
                
                while article_index < len(articles):
                    article = articles[article_index]
                    try:
                        headline = article.find('h2').get_text(strip=True)
                        article_url = article.find('a')['href']
                        article_id = article.get("data-id", None)
                        article_response = requests.get(article_url)
                        article_response.raise_for_status()
                        content_soup = BeautifulSoup(article_response.text, "html.parser")
                        paragraphs = content_soup.find('div', class_='content-area').find_all('p')
                        body_content = " ".join(p.get_text(strip=True) for p in paragraphs)

                        geo_data['article_id'].append(article_id)
                        geo_data['headline'].append(headline)
                        geo_data['url'].append(article_url)
                        geo_data['body_content'].append(body_content)
                        geo_data['category_label'].append(section)

                    except Exception as e:
                        print(f"\t--> Failed to scrape article in '{section}': {e}")

                    article_index += 1

            except Exception as e:
                print(f"Failed to fetch '{section}', error: {e}")

            section_index += 1

        return pd.DataFrame(geo_data)
if __name__ == "__main__":
    # Initialize the NewsScraper instance
    scraper = NewsScraper()
    
    # Test scraping articles from Jang
    print("Testing Jang Articles Scraping...")
    jang_articles_df = scraper.get_jang_articles()
    print(f"Scraped {len(jang_articles_df)} articles from Jang.")
    print(jang_articles_df.head())

    # Test scraping articles from Geo
    print("\nTesting Geo Articles Scraping...")
    geo_articles_df = scraper.get_geo_articles()
    print(f"Scraped {len(geo_articles_df)} articles from Geo.")
    print(geo_articles_df.head())


In [5]:
# Initialize the NewsScraper instance
scraper = NewsScraper()


In [7]:
# Test scraping from Jang
print("\nScraping from Jang:\n")
jang_df = scraper.get_jang_articles()
print(jang_df.head())  # Displaying the first few rows of the DataFrame


Scraping from Jang:

Scraping 'entertainment' section...
Scraping 'business' section...
Scraping 'sports' section...
Scraping 'technology' section...
Scraping 'world' section...
     id                                              title  \
0  None  آسکر ایوارڈ یافتہ بھارتی موسیقار اے آر رحمٰن ...   
1  None  کمی کے باوجود اینٹی اسموگ مہم جاری رکھنے کا اعلان   
2  None  بچوں کو مشاورت میں شریک کرنے کی تہذیب کو فروغ ...   
3  None  گزشتہ حکومت پوچھنے پر بھی توشہ خانے کی تفصیلات...   
4  None  ڈونلڈ ٹرمپ نے ڈبلیو ڈبلیو ای کی سابق سی ای او ...   

                               link  \
0  https://jang.com.pk/news/1412845   
1  https://jang.com.pk/news/1412843   
2  https://jang.com.pk/news/1412842   
3  https://jang.com.pk/news/1412841   
4  https://jang.com.pk/news/1412840   

                                             content     gold_label  
0   اسکر ایوارڈ یافتہ بھارتی معروف موسیقار و گلوک...  entertainment  
1   پنجاب کی سینئر وزیر مریم اورنگزیب نے کہا ہے ک...  entertainment  
2  

In [9]:
jang_df

,id,title,link,content,gold_label
0,None,آسکر ایوارڈ یافتہ بھارتی موسیقار اے آر رحمٰن ...,https://jang.com.pk/news/1412845,اسکر ایوارڈ یافتہ بھارتی معروف موسیقار و گلوک...,entertainment
1,None,کمی کے باوجود اینٹی اسموگ مہم جاری رکھنے کا اعلان,https://jang.com.pk/news/1412843,پنجاب کی سینئر وزیر مریم اورنگزیب نے کہا ہے ک...,entertainment
2,None,بچوں کو مشاورت میں شریک کرنے کی تہذیب کو فروغ ...,https://jang.com.pk/news/1412842,وزیرِ اعلیٰ سندھ مراد علی شاہ نے بچوں کے عالم...,entertainment
3,None,گزشتہ حکومت پوچھنے پر بھی توشہ خانے کی تفصیلات...,https://jang.com.pk/news/1412841,بانئ پی ٹی آئی کی توشہ خانہ کیس ٹو میں ضمانت ...,entertainment
4,None,ڈونلڈ ٹرمپ نے ڈبلیو ڈبلیو ای کی سابق سی ای او ...,https://jang.com.pk/news/1412840,ڈونلڈ ٹرمپ نے ورلڈ ریسلنگ انٹرٹینمنٹ (ڈبلیو ڈ...,entertainment
...,...,...,...,...,...
505,None,اے ڈی بی قرض پروگرام کی رقم موسمیاتی تبدیلی سے...,https://jang.com.pk/news/1412537,اقتصادی امور کے ترجمان کا کہنا ہے کہ پاکستان...,world
506,None,گورنر پنجاب سلیم حیدر خان کا طلباء یونینز کی ب...,https://jang.com.pk/news/1412536,گورنر پنجاب سلیم حیدر خان کا کہنا ہے کہ طلباء...,world
507,None,سرگودھا: پشاور جانیوالی مسافر بس میں آگ لگ گئی,https://jang.com.pk/news/1412535,سرگودھا کے علاقے چناب پل کے قریب مسافر بس میں...,world
508,None,بھارت: انتہائی کم قیمت پر بیچا گیا ادرک لہسن ک...,https://jang.com.pk/news/1412534,بھارت کی ریاست تلنگانہ میں انتہائی کم قیمت پر...,world


In [41]:
# Test scraping from GEO
print("\nScraping from GEO:\n")
geo_df = scraper.get_geo_articles(max_pages=2)
print(geo_df.head())  # Displaying the first few rows of the DataFrame



Scraping from GEO:

Scraping 'entertainment' section...
Scraping 'business' section...
Scraping 'sports' section...
Scraping 'science-technology' section...
Scraping 'world' section...
    id                                              title  \
0  510  عدنان صدیقی کی برطانوی بادشاہ سے ملاقات، شاہ چ...   
1  511  سعودی دارالحکومت ریاض میں فیشن شو، معروف گلوکا...   
2  512  پاکستانی مداح نے کنسرٹ میں میکا سنگھ کو کروڑوں...   
3  513  سدھو نے  کپل شرما شو چھوڑنے کی اصل وجہ 5 سال ب...   
4  514  امیتابھ بچن بیٹے ابھیشیک کو اپنے شو میں بلاکر ...   

                                 link  \
0  https://urdu.geo.tv/latest/387006-   
1  https://urdu.geo.tv/latest/386962-   
2  https://urdu.geo.tv/latest/386939-   
3  https://urdu.geo.tv/latest/386888-   
4  https://urdu.geo.tv/latest/386887-   

                                             content     gold_label  
0  پاکستانی شوبز  انڈسٹری کے سینئر اداکار عدنان ص...  entertainment  
1    سعودی عرب کے دارالحکومت ریاض میں فیشن شو کا ...  entert

In [43]:
geo_df

,id,title,link,content,gold_label
0,510,عدنان صدیقی کی برطانوی بادشاہ سے ملاقات، شاہ چ...,https://urdu.geo.tv/latest/387006-,پاکستانی شوبز انڈسٹری کے سینئر اداکار عدنان ص...,entertainment
1,511,سعودی دارالحکومت ریاض میں فیشن شو، معروف گلوکا...,https://urdu.geo.tv/latest/386962-,سعودی عرب کے دارالحکومت ریاض میں فیشن شو کا ...,entertainment
2,512,پاکستانی مداح نے کنسرٹ میں میکا سنگھ کو کروڑوں...,https://urdu.geo.tv/latest/386939-,بالی وڈ کے معروف گلوکار میکا سنگھ کو کنسرٹ می...,entertainment
3,513,سدھو نے کپل شرما شو چھوڑنے کی اصل وجہ 5 سال ب...,https://urdu.geo.tv/latest/386888-,بھارتی ریاست پنجاب کے وزیر و سابق کرکٹر نووجوت...,entertainment
4,514,امیتابھ بچن بیٹے ابھیشیک کو اپنے شو میں بلاکر ...,https://urdu.geo.tv/latest/386887-,بالی وڈ کے بگ بی امیتابھ بچن ’کون بنے گا کروڑ ...,entertainment
...,...,...,...,...,...
295,805,حطیم میں نوافل کی ادائیگی کے اوقات جاری,https://urdu.geo.tv/latest/386429-,مکہ مکرمہ: خانہ کعبہ کی حدود حطیم میں نوافل ...,world
296,806,غزہ میں مزاحمت کاروں کے حملے میں 4 اسرائیلی فو...,https://urdu.geo.tv/latest/386427-,تل ابیب: غزہ میں فلسطینی مزاحمت کاروں کے حمل...,world
297,807,ٹرمپ کے ’47 ویں‘ امریکی صدر بننے کی راہ میں کم...,https://urdu.geo.tv/latest/386411-,امریکا میں 5 نومبر کو ہونے والے صدارتی انتخابا...,world
298,808,روس کا آبادی میں اضافے کیلئے اقدامات پر غور، ...,https://urdu.geo.tv/latest/386410-,روس نے ملک کی آبادی میں اضافے کیلئے خصوصی وزا...,world


In [45]:
geo_df.to_csv("geo_scraped_data.csv", index=False)
express_df.to_csv("express_scraped_data.csv", index=False)
jang_df.to_csv("jang_scraped_data.csv", index=False)
articles_df.to_csv("ary_scraped_data.csv",index=False)
dunya_df.to_csv("dunya_scraped_data.csv",index = False)
print("\nData has been successfully saved to separate CSV files: 'geo_scraped_data.csv','dunya_scraped_data', 'express_scraped_data.csv','ary_scraped_data' and 'jang_scraped_data.csv'.")


Data has been successfully saved to separate CSV files: 'geo_scraped_data.csv', 'express_scraped_data.csv', and 'jang_scraped_data.csv'.
